<a href="https://colab.research.google.com/github/163093-Shubhangi/DLNLP_project/blob/main/Credit_Card.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split

In [ ]:
import os
for dirname, _, filenames in os.walk('/content/drive/MyDrive/Credit_Card'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
pathToFolder = "/content/drive/MyDrive/Credit_Card/"

In [ ]:
path =  "/content/drive/MyDrive/Credit_Card/application_record.csv"

In [ ]:
df_app = pd.read_csv(path)
df_app.head(3)


,ID,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS
0,5008804,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.0
1,5008805,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.0
2,5008806,M,Y,Y,0,112500.0,Working,Secondary / secondary special,Married,House / apartment,-21474,-1134,1,0,0,0,Security staff,2.0


In [ ]:
df_cred = pd.read_csv('/content/drive/MyDrive/Credit_Card/credit_record.csv')
df_cred.head(3)

,ID,MONTHS_BALANCE,STATUS
0,5001711,0,X
1,5001711,-1,0
2,5001711,-2,0


In [ ]:
final_df = df_app.merge(df_cred, on=['ID'],how='inner')
final_df.head()
# Merge both application_record and credit record on ID

,ID,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,MONTHS_BALANCE,STATUS
0,5008804,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.0,0,C
1,5008804,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.0,-1,C
2,5008804,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.0,-2,C
3,5008804,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.0,-3,C
4,5008804,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.0,-4,C


In [ ]:
final_df.isnull().sum()

ID                          0
CODE_GENDER                 0
FLAG_OWN_CAR                0
FLAG_OWN_REALTY             0
CNT_CHILDREN                0
AMT_INCOME_TOTAL            0
NAME_INCOME_TYPE            0
NAME_EDUCATION_TYPE         0
NAME_FAMILY_STATUS          0
NAME_HOUSING_TYPE           0
DAYS_BIRTH                  0
DAYS_EMPLOYED               0
FLAG_MOBIL                  0
FLAG_WORK_PHONE             0
FLAG_PHONE                  0
FLAG_EMAIL                  0
OCCUPATION_TYPE        240048
CNT_FAM_MEMBERS             0
MONTHS_BALANCE              0
STATUS                      0
dtype: int64

In [ ]:
final_df.shape

(777715, 20)

In [ ]:
# drop null values, axis=0
final_df.dropna(inplace=True)
final_df.shape

(537667, 20)

In [ ]:
#1,2,3,4 and 5 are bad clients, X and C are good clients and 0 consider as neutral (not bad and not good) so let's take 0 also as good client.
# replacing the status values as good -> 1, bad -> 0

final_df.STATUS = final_df.STATUS.map({"X":1,"C":1, "0":1, "1":0, "2":0, "3":0, "4":0, "5":0})
final_df.sample(5)

,ID,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,MONTHS_BALANCE,STATUS
316331,5061652,M,N,Y,1,157500.0,State servant,Incomplete higher,Married,House / apartment,-8948,-1751,1,0,0,0,Core staff,3.0,0,1
28776,5010511,F,Y,Y,1,157500.0,Commercial associate,Secondary / secondary special,Married,House / apartment,-13830,-4239,1,0,0,0,Laborers,3.0,-7,1
468767,5091737,F,Y,Y,0,243000.0,Working,Secondary / secondary special,Single / not married,House / apartment,-14716,-2057,1,0,0,0,Sales staff,1.0,-41,1
183307,5038714,F,N,Y,0,238500.0,Working,Secondary / secondary special,Married,House / apartment,-22246,-14536,1,0,0,0,High skill tech staff,2.0,-11,1
487953,5095266,F,Y,Y,0,126000.0,Commercial associate,Secondary / secondary special,Married,House / apartment,-10302,-734,1,0,0,0,Core staff,2.0,-32,1


In [ ]:
final_df.isnull().sum()

ID                     0
CODE_GENDER            0
FLAG_OWN_CAR           0
FLAG_OWN_REALTY        0
CNT_CHILDREN           0
AMT_INCOME_TOTAL       0
NAME_INCOME_TYPE       0
NAME_EDUCATION_TYPE    0
NAME_FAMILY_STATUS     0
NAME_HOUSING_TYPE      0
DAYS_BIRTH             0
DAYS_EMPLOYED          0
FLAG_MOBIL             0
FLAG_WORK_PHONE        0
FLAG_PHONE             0
FLAG_EMAIL             0
OCCUPATION_TYPE        0
CNT_FAM_MEMBERS        0
MONTHS_BALANCE         0
STATUS                 0
dtype: int64

In [ ]:
final_df.head(4)

,ID,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,MONTHS_BALANCE,STATUS
31,5008806,M,Y,Y,0,112500.0,Working,Secondary / secondary special,Married,House / apartment,-21474,-1134,1,0,0,0,Security staff,2.0,0,1
32,5008806,M,Y,Y,0,112500.0,Working,Secondary / secondary special,Married,House / apartment,-21474,-1134,1,0,0,0,Security staff,2.0,-1,1
33,5008806,M,Y,Y,0,112500.0,Working,Secondary / secondary special,Married,House / apartment,-21474,-1134,1,0,0,0,Security staff,2.0,-2,1
34,5008806,M,Y,Y,0,112500.0,Working,Secondary / secondary special,Married,House / apartment,-21474,-1134,1,0,0,0,Security staff,2.0,-3,1


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# replace M & F as 0 & 1
final_df['CODE_GENDER'].replace('M',0,inplace=True) #male -> 0
final_df['CODE_GENDER'].replace('F',1,inplace=True)#female -> 1
final_df['FLAG_OWN_CAR'].replace('Y',0,inplace=True)
final_df['FLAG_OWN_CAR'].replace('N',1,inplace=True)
final_df['FLAG_OWN_REALTY'].replace('Y',0,inplace=True)
final_df['FLAG_OWN_REALTY'].replace('N',1,inplace=True)
final_df.head(4)

,ID,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,MONTHS_BALANCE,STATUS
31,5008806,0,0,0,0,112500.0,Working,Secondary / secondary special,Married,House / apartment,-21474,-1134,1,0,0,0,Security staff,2.0,0,1
32,5008806,0,0,0,0,112500.0,Working,Secondary / secondary special,Married,House / apartment,-21474,-1134,1,0,0,0,Security staff,2.0,-1,1
33,5008806,0,0,0,0,112500.0,Working,Secondary / secondary special,Married,House / apartment,-21474,-1134,1,0,0,0,Security staff,2.0,-2,1
34,5008806,0,0,0,0,112500.0,Working,Secondary / secondary special,Married,House / apartment,-21474,-1134,1,0,0,0,Security staff,2.0,-3,1


In [ ]:
final_df.NAME_INCOME_TYPE.value_counts()

Working                 332868
Commercial associate    151412
State servant            52733
Pensioner                  332
Student                    322
Name: NAME_INCOME_TYPE, dtype: int64

In [ ]:
def income_tans(df,col):
    result_col = []
    for i in df[col]:
        if i == 'Working':
            result_col.append('Working')
        elif i == 'Commercial associate':
            result_col.append('Commercial associate')
        elif i == 'State servant':
            result_col.append('State servant')
        else:
            result_col.append('others')
    df[col] = result_col
    return df


income_tans(final_df,'NAME_INCOME_TYPE')
final_df.NAME_INCOME_TYPE.value_counts()

Working                 332868
Commercial associate    151412
State servant            52733
others                     654
Name: NAME_INCOME_TYPE, dtype: int64

In [ ]:
final_df.NAME_EDUCATION_TYPE.value_counts()

Secondary / secondary special    358317
Higher education                 153770
Incomplete higher                 20590
Lower secondary                    4556
Academic degree                     434
Name: NAME_EDUCATION_TYPE, dtype: int64

In [ ]:
final_df.NAME_EDUCATION_TYPE = final_df.NAME_EDUCATION_TYPE.str.split('/').str[0]
final_df.NAME_EDUCATION_TYPE.value_counts()

Secondary            358317
Higher education     153770
Incomplete higher     20590
Lower secondary        4556
Academic degree         434
Name: NAME_EDUCATION_TYPE, dtype: int64

In [ ]:
final_df.NAME_FAMILY_STATUS.value_counts()

Married                 384003
Single / not married     65944
Civil marriage           44083
Separated                31394
Widow                    12243
Name: NAME_FAMILY_STATUS, dtype: int64

In [ ]:
final_df.NAME_FAMILY_STATUS = final_df.NAME_FAMILY_STATUS.str.split('/').str[0]
final_df.NAME_FAMILY_STATUS.value_counts()

Married           384003
Single             65944
Civil marriage     44083
Separated          31394
Widow              12243
Name: NAME_FAMILY_STATUS, dtype: int64

In [ ]:
final_df.head(3)

,ID,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,MONTHS_BALANCE,STATUS
31,5008806,0,0,0,0,112500.0,Working,Secondary,Married,House / apartment,-21474,-1134,1,0,0,0,Security staff,2.0,0,1
32,5008806,0,0,0,0,112500.0,Working,Secondary,Married,House / apartment,-21474,-1134,1,0,0,0,Security staff,2.0,-1,1
33,5008806,0,0,0,0,112500.0,Working,Secondary,Married,House / apartment,-21474,-1134,1,0,0,0,Security staff,2.0,-2,1


In [ ]:
final_df.AMT_INCOME_TOTAL = final_df.AMT_INCOME_TOTAL/1000
final_df.AMT_INCOME_TOTAL = final_df.AMT_INCOME_TOTAL.astype('int')
final_df.head(3)

,ID,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,MONTHS_BALANCE,STATUS
31,5008806,0,0,0,0,112,Working,Secondary,Married,House / apartment,-21474,-1134,1,0,0,0,Security staff,2.0,0,1
32,5008806,0,0,0,0,112,Working,Secondary,Married,House / apartment,-21474,-1134,1,0,0,0,Security staff,2.0,-1,1
33,5008806,0,0,0,0,112,Working,Secondary,Married,House / apartment,-21474,-1134,1,0,0,0,Security staff,2.0,-2,1


In [ ]:
final_df['Age']=-(final_df['DAYS_BIRTH'])//365

final_df['employee_from_years']=-(final_df['DAYS_EMPLOYED'])//365

final_df.drop(columns=('DAYS_BIRTH'), inplace=True)

final_df.drop(columns=('DAYS_EMPLOYED'), inplace=True)
final_df.head(4)

,ID,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,FLAG_MOBIL,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,MONTHS_BALANCE,STATUS,Age,employee_from_years
31,5008806,0,0,0,0,112,Working,Secondary,Married,House / apartment,1,0,0,0,Security staff,2.0,0,1,58,3
32,5008806,0,0,0,0,112,Working,Secondary,Married,House / apartment,1,0,0,0,Security staff,2.0,-1,1,58,3
33,5008806,0,0,0,0,112,Working,Secondary,Married,House / apartment,1,0,0,0,Security staff,2.0,-2,1,58,3
34,5008806,0,0,0,0,112,Working,Secondary,Married,House / apartment,1,0,0,0,Security staff,2.0,-3,1,58,3


In [ ]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 537667 entries, 31 to 777714
Data columns (total 20 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   ID                   537667 non-null  int64  
 1   CODE_GENDER          537667 non-null  int64  
 2   FLAG_OWN_CAR         537667 non-null  int64  
 3   FLAG_OWN_REALTY      537667 non-null  int64  
 4   CNT_CHILDREN         537667 non-null  int64  
 5   AMT_INCOME_TOTAL     537667 non-null  int64  
 6   NAME_INCOME_TYPE     537667 non-null  object 
 7   NAME_EDUCATION_TYPE  537667 non-null  object 
 8   NAME_FAMILY_STATUS   537667 non-null  object 
 9   NAME_HOUSING_TYPE    537667 non-null  object 
 10  FLAG_MOBIL           537667 non-null  int64  
 11  FLAG_WORK_PHONE      537667 non-null  int64  
 12  FLAG_PHONE           537667 non-null  int64  
 13  FLAG_EMAIL           537667 non-null  int64  
 14  OCCUPATION_TYPE      537667 non-null  object 
 15  CNT_FAM_MEMBERS 

In [ ]:
final_df.head(4)

,ID,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,FLAG_MOBIL,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,MONTHS_BALANCE,STATUS,Age,employee_from_years
31,5008806,0,0,0,0,112,Working,Secondary,Married,House / apartment,1,0,0,0,Security staff,2.0,0,1,58,3
32,5008806,0,0,0,0,112,Working,Secondary,Married,House / apartment,1,0,0,0,Security staff,2.0,-1,1,58,3
33,5008806,0,0,0,0,112,Working,Secondary,Married,House / apartment,1,0,0,0,Security staff,2.0,-2,1,58,3
34,5008806,0,0,0,0,112,Working,Secondary,Married,House / apartment,1,0,0,0,Security staff,2.0,-3,1,58,3


In [ ]:
final_df.drop(columns=['ID','CNT_CHILDREN','FLAG_MOBIL','FLAG_WORK_PHONE','FLAG_EMAIL'], inplace=True)
final_df.sample(10)

,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,FLAG_PHONE,OCCUPATION_TYPE,CNT_FAM_MEMBERS,MONTHS_BALANCE,STATUS,Age,employee_from_years
455812,0,0,0,405,Working,Higher education,Married,House / apartment,1,Managers,2.0,-10,1,28,7
462545,1,1,0,135,Working,Secondary,Single,House / apartment,0,Laborers,1.0,-33,1,48,5
641529,1,1,0,157,Commercial associate,Higher education,Married,House / apartment,0,High skill tech staff,4.0,-38,1,37,0
37155,1,1,0,90,Working,Secondary,Married,House / apartment,0,Medicine staff,2.0,-12,1,54,0
445353,1,1,1,157,State servant,Higher education,Married,House / apartment,1,Core staff,2.0,-12,1,31,2
456753,1,1,0,270,Commercial associate,Higher education,Married,House / apartment,0,Secretaries,4.0,-20,1,42,6
512793,0,1,1,270,Commercial associate,Higher education,Married,House / apartment,1,Core staff,2.0,-6,1,35,0
714642,1,1,1,180,Working,Secondary,Married,House / apartment,1,Accountants,2.0,-26,1,50,12
18224,0,0,1,427,Commercial associate,Higher education,Married,House / apartment,0,Managers,4.0,-13,1,41,17
249737,0,0,1,346,Commercial associate,Secondary,Married,House / apartment,0,Laborers,2.0,-1,1,24,4


In [ ]:
# one hot encoding
def one_hot_encode(df, columns):
    for col in columns:
        one_hot = pd.get_dummies(df[col], drop_first=True)
        df = pd.concat([df, one_hot], axis=1)
        df = df.drop([col], axis=1)
    return df

categories=['NAME_INCOME_TYPE','NAME_EDUCATION_TYPE','NAME_FAMILY_STATUS','NAME_HOUSING_TYPE','OCCUPATION_TYPE']
final_df = one_hot_encode(final_df,categories)
final_df.head(5)

In [ ]:
X = final_df.drop(columns=('STATUS'),axis=1)
y = final_df.STATUS
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=2)
print('X_train :', X_train.shape)
print('X_test :', X_test.shape)
print('y_train :', y_train.shape)
print('y_test :', y_test.shape)

In [ ]:
ss = StandardScaler()

In [ ]:
ss_fit = ss.fit_transform(X)

In [ ]:
X_train,X_test, y_train,y_test = train_test_split(X, y, test_size = 0.25)

In [ ]:
X_train.shape
X_test.shape

(134417, 14)

In [ ]:
mm = MinMaxScaler()
mm.fit(X_train)

In [ ]:
model = tf.keras.Sequential()

In [ ]:
model.add(layers.Input( shape = (42,) ) )

In [ ]:
model.add(tf.keras.layers.Dense(500, activation = 'relu'))

In [ ]:
model.add(tf.keras.layers.Dense(2, activation = 'relu'))

In [ ]:
model.fit(X,y, epochs = 100 )

RuntimeError: ignored

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 25)                225       
                                                                 
 dense_1 (Dense)             (None, 1)                 26        
                                                                 
Total params: 251
Trainable params: 251
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.compile(loss = "mae")

In [ ]:
history1 = model.fit(X_train,y_train,
                    epochs = 70,
                    validation_data = (X_test, y_test)
                   )

In [ ]:
model.evaluate(X_test,y_test)